In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.morphology import skeletonize, remove_small_objects
##creating a circular mask so any stuff outside the circle will be disregarded   
z = np.zeros((1078,1088))
# specify circle parameters: centre ij and radius
ci,cj=540,539
cr=539
# Create index arrays to z
I,J=np.meshgrid(np.arange(z.shape[0]),np.arange(z.shape[1]))
# calculate distance of all points to centre
dist=np.sqrt((I-ci)**2+(J-cj)**2)
# Assign value of 1 to those points where dist<cr:
z[np.where(dist<cr)]=1

output = np.zeros((25,2))

for i in range(25):
    fname = "path%02d.tif" % (1+i)
    #print(fname)
    
    original_img = cv2.imread(fname,0)

    (h,w)=np.shape(original_img)
    img = original_img[0:h, 0:w]
    #plt.imshow(img, cmap = 'gray')
    #equalizing the histogram
    equ = cv2.equalizeHist(img)
    
     #This sction blurs, thresholds, and skeletonizes the image
    #It uses an average blur and an adaptive (local) mean threshold
    #Remove_small_objects gets rid of any unconnected dots or lines
    ablur = cv2.blur(equ,(17, 17))
    a_mthres = cv2.adaptiveThreshold(ablur,1,cv2.ADAPTIVE_THRESH_MEAN_C,\
                cv2.THRESH_BINARY,171,4) 
    adaptiveskeleton = skeletonize(a_mthres)
    clskel=remove_small_objects(adaptiveskeleton, min_size=250, connectivity=3)
    cleanskel=clskel*z

    #the original image values (integers) become floats
    #Dividing by 255 normalizes the floats; now they are all between 0 and 1
    floatimg=img.astype(float)
    normimg=floatimg/255

    #skeleton is a float, rgbpic is a RGB image with the dimensions of the cropped diatom image
    #the values of R,G, and B are defined later
    floatskel=cleanskel.astype(float)
    (x,y)=np.shape(img)
    rgbpic=np.zeros((x,y,3))
    
     # this section finds the number of neighbors to each point in the skeleton
    # line ends are points that have only one neighbor
    # we want to get rid of these
    # keep looping over this section until the number of line ends equals zero
    nLineEnds = 1
    while (nLineEnds>0):
        #blur the skeleton with 3x3; this gives the average value in a 3x3 portion of the skeleton
        # 9 times the average - 1 should be the number of neighbors
        skelblur = cv2.blur(floatskel,(3,3))
        neighbors=((9*skelblur)-1)

        #making sure edges always have values
        #any skeleton pixels within 'a' pixels from the edge will not be eroded
        edges=np.zeros((x,y))
        a=15
        edges[0:a,:]=np.ones((a,y))
        edges[x-a:x,:]=np.ones((a,y))
        edges[:,0:a]=np.ones((x,a))
        edges[:,y-a:y]=np.ones((x,a))

        #removing the end pixel if a skeleton pixel has 1 neighbor
        neighbors=(neighbors+5*edges)*floatskel
        line_ends = (neighbors==1)
        float_line_ends = line_ends.astype(float)
        floatskel=floatskel-line_ends
        nLineEnds=np.sum(line_ends)
    #plt.imshow(neighbors, interpolation='sinc')
    
    #reading in the dilated correct skeleton to assess su
    correctskeleton=cv2.imread("dilation1.tif", 0)

    # total length of skeleton
    #print("total length:", np.sum(floatskel))
    
    #for the number of pixels in the right place
    #print("pixels in the right place:", np.sum(correctskeleton*floatskel))
    
    output[i,0] = np.sum(floatskel)
    output[i,1] = np.sum(correctskeleton*floatskel)
    
print(output)
    

[[72042.   853.]
 [71327.  1533.]
 [39228.  2260.]
 [24569.  2714.]
 [15717.  3300.]
 [13777.  3876.]
 [11974.  4402.]
 [11491.  4573.]
 [11463.  5133.]
 [11022.  5407.]
 [11045.  5665.]
 [10960.  6004.]
 [10923.  6347.]
 [10935.  6515.]
 [10914.  6698.]
 [10840.  6624.]
 [10896.  6701.]
 [10852.  7138.]
 [10818.  6991.]
 [10815.  7032.]
 [10818.  7098.]
 [10810.  7164.]
 [10803.  7222.]
 [10804.  7196.]
 [10780.  7280.]]


In [6]:
np.savetxt('outputdil1.csv',output,delimiter=',')